In [0]:
https://langchain-ai.github.io/langgraph/how-tos/graph-api/#add-runtime-configuration

In [0]:
builder.add_node("position_refined", position, metadata={"__my_": "his"})
builder.add_node("position", position, metadata={"__my__": "yours"})
....
def position(state: State, config: RunnableConfig):
  my = config["metadata"]["__my__"]

In [0]:
from typing import Dict, Any
from langgraph.graph import END, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_core.runnables import RunnableConfig
from langchain_core.runnables.base import RunnableBinding
from typing_extensions import TypedDict
from databricks_langchain import ChatDatabricks

from agents.rag.tools import documentation_search_tool

claude = ChatDatabricks(endpoint="databricks-claude-sonnet-4")
llama = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct")

tools = [documentation_search_tool]
claude_with_tools = claude.bind_tools(tools, tool_choice="auto")
llama_with_tools = llama.bind_tools(tools, tool_choice="auto")
tool_node = ToolNode(tools=tools)

class LlamaSchema(TypedDict):
    model_with_tools: RunnableBinding = llama_with_tools

class ClaudeSchema(TypedDict):
    model_with_tools: RunnableBinding = claude_with_tools

class LLMConfig(TypedDict):
  model_with_tools: RunnableBinding

In [0]:
system_message = """You are a trusted assistant capable of searching Databricks and Apache Spark documentation. Based on the users most recent question and relevent conversation history, call the documentation search tools you have access to. It's possible that only part of the user's question and recent conversation history is relevent to your area of expertise. Use your judgment to identify which part's of the user's question can be used to retrieved relevent documentation. Make sure your final answer is grounded on the documents returned by the tool."""


def chatbot(state: MessagesState, config: RunnableConfig):
  """
  Retrieve relevent documents based on the user's question and conversation
  history. Answer the user's quesiton based on the documentation.
  """
  messages = [{"role": "system", "content": system_message}] + state["messages"]
  print(config["configurable"])
  model_with_tools = config["configurable"]["model_with_tools"]
  response = model_with_tools.invoke(messages)
  return {"messages": [response]}


def route_tools(state: MessagesState):
  """
  A router that determines if tools should be called or a final
  answer returned to the user
  """
  ai_message = state["messages"][-1]
  
  if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
  
  return END

from langgraph.graph import StateGraph, START, END
from langgraph.graph import END, MessagesState


graph_builder = StateGraph(MessagesState, config_schema=LLMConfig)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges("chatbot", route_tools,
    {"tools": "tools", 
      END: END}
  ) 
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("tools", "chatbot")
rag_agent = graph_builder.compile(name="rag_agent")

messages = {"messages": [{"role": "user", "content": "Does Apache Spark support streaming?"}]}
for event in rag_agent.stream(messages, {"configurable": {"model_with_tools": claude_with_tools}}, stream_mode= "updates"):
  print(event)

In [0]:
messages = {"messages": [{"role": "user", "content": "Does Apache Spark support streaming?"}]}
rag_agent.invoke(messages)